In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
def direction(val):
    val1 = Conversion().convert_int256_bytes_to_int(val, signed=False) 
    val2 = Conversion().convert_int256_bytes_to_int(val, signed=True)
    return val1 != val2

In [3]:
chain = Net.POLYGON
platform = Platform.UNIV3
contract = JSONContract.UniswapV3Factory

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [4]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoad().get_contract(web3, abi_path)
pool_created_filt = Filter.create_filter(address=None, event_types=[Pair.events.PoolCreated])  # Listen events from any smart contract

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [6]:
start = start - 10000

In [7]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=pool_created_filt)

In [8]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    event_signature, token0, token1, fee = evt["topics"]
    args = Conversion().decode_data(evt["data"])

    token0_address = Conversion().convert_uint256_string_to_address(token0)
    token1_address = Conversion().convert_uint256_string_to_address(token1)
    token0 = Token().fetch_erc20_details(web3, token0_address, raise_on_error=False)
    token1 = Token().fetch_erc20_details(web3, token1_address, raise_on_error=False)
    
    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = token0_address
    event['details']['token1'] = token1_address
    event['details']['token0_symbol'] = token0.symbol
    event['details']['token1_symbol'] = token1.symbol  
    event['details']['fee'] = Conversion().convert_uint256_string_to_int(fee) 
 
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:61,374,970 tx:0x02099033f2f24e296aa903ade95af4fe5e795266866b895a95eaac16e84d39c2
Swap at block:61,375,210 tx:0xbc756ad8d7247ef56d5c3ca5cd0b4cf02b16e6bfde40d65167c281ca4a0740fd
Swap at block:61,375,705 tx:0x34798f4e9bc248232f7a2decd5eae301626910bfb28c7eceb5f22196be493256
Swap at block:61,377,341 tx:0x2035b01430d60413849a15aaddfd526c9a3981610b9be24c232451f459328559
Swap at block:61,381,760 tx:0x719ed60496788b0ee7726bb370ed041b72820bfb7a33d3fcdb54ece3612b72a2
.


In [9]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x02099033f2f24e296aa903ade95af4fe5e795266866b...,61374970,1725365742,{'web3_type': <class 'web3._utils.datatypes.Po...
1,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x917933899c6a5f8e37f31e19f92cdbff7e8ff0e2,0xbc756ad8d7247ef56d5c3ca5cd0b4cf02b16e6bfde40...,61375210,1725366265,{'web3_type': <class 'web3._utils.datatypes.Po...
2,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x34798f4e9bc248232f7a2decd5eae301626910bfb28c...,61375705,1725367381,{'web3_type': <class 'web3._utils.datatypes.Po...
3,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x2035b01430d60413849a15aaddfd526c9a3981610b9b...,61377341,1725370994,{'web3_type': <class 'web3._utils.datatypes.Po...
4,polygon,uniswapv3factory,poolcreated,uniswap_v3,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x719ed60496788b0ee7726bb370ed041b72820bfb7a33...,61381760,1725380635,{'web3_type': <class 'web3._utils.datatypes.Po...


In [10]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv3factory',
  'type': 'poolcreated',
  'platform': 'uniswap_v3',
  'address': '0x1f98431c8ad98523631ae4a59f267346ea31f984',
  'tx_hash': '0x02099033f2f24e296aa903ade95af4fe5e795266866b895a95eaac16e84d39c2',
  'blk_num': 61374970,
  'timestamp': 1725365742,
  'details': {'web3_type': web3._utils.datatypes.PoolCreated,
   'token0': '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270',
   'token1': '0xde96e5281b50bD954b250331058B3b768f7Dbb77',
   'token0_symbol': 'WMATIC',
   'token1_symbol': 'ACCS',
   'fee': 3000}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv3factory',
  'type': 'poolcreated',
  'platform': 'uniswap_v3',
  'address': '0x917933899c6a5f8e37f31e19f92cdbff7e8ff0e2',
  'tx_hash': '0xbc756ad8d7247ef56d5c3ca5cd0b4cf02b16e6bfde40d65167c281ca4a0740fd',
  'blk_num': 61375210,
  'timestamp': 1725366265,
  'details': {'web3_type': web3._utils.datatypes.PoolCreated,
   'token0': '0x6D6AE4cD70ce1240E905F43f6C11882a3A6dE139',
   'token1': '0